# Data Ingestion Module

In [1]:
import os
!pwd

/teamspace/studios/this_studio/text-summarizer/research


In [2]:
os.chdir('../')
!pwd

/teamspace/studios/this_studio/text-summarizer


### Basic Configuration

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_uri: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
from src.textSummarizer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.textSummarizer.utils.common import read_yaml, create_directories

### Configuration updates

In [5]:
class CofigurationManager:
    def __init__(self, config_path=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_filepath)

        # create_directories([self.config.artifact_root])
        os.makedirs(self.config.artifact_root, exist_ok=True)
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        try:
            config = self.config.data_ingestion
            
            # Ensure that the root directory for data ingestion exists
            # create_directories([config.root_dir])
            os.makedirs(config.root_dir, exist_ok=True)
            return DataIngestionConfig(
                root_dir=config.root_dir,
                source_uri=config.source_uri,
                local_data_file=config.local_data_file,
                unzip_dir=config.unzip_dir
            )
        except AttributeError as e:
            logger.error(f"Missing data ingestion configuration: {e}")
            return None
        except Exception as e:
            logger.error(f"Error in creating DataIngestionConfig: {e}")
            return None


In [6]:
import os
import urllib.request as request
import zipfile
from src.textSummarizer.logging import create_logger

### Components

In [7]:
import os
from urllib import request

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        self.logger = create_logger(__name__)
        
    def download_data(self):
        try:
            # Check if the directory exists
            if not os.path.exists(self.config.local_data_file):
                self.logger.info(f"Creating the directory {self.config.local_data_file}")
                os.makedirs(self.config.local_data_file)

            # Define the file path where the data will be saved
            zip_file_path = os.path.join(self.config.local_data_file, "summarizer-data.zip")  # Specify the file name

            if not os.path.exists(zip_file_path):  # Check if the file already exists
                self.logger.info(f"Downloading data from {self.config.source_uri}")
                filename, headers = request.urlretrieve(
                    url=self.config.source_uri, 
                    filename=zip_file_path)  # Use the full path with the file name
            else:
                self.logger.info(f"File {zip_file_path} already exists")
            
        except Exception as e:
            self.logger.error(f"Error downloading data from {self.config.source_uri}")
            self.logger.error(e)
            raise e
    
    def extract_zip_file(self):
        try:
            # Ensure the extraction directory exists
            os.makedirs(self.config.unzip_dir, exist_ok=True)
            
            # Specify the correct file path for the downloaded zip file
            zip_file_path = os.path.join(self.config.local_data_file, "summarizer-data.zip")  # Correct zip file path

            # Use the correct file path to extract the zip file
            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                zip_ref.extractall(self.config.unzip_dir)
            self.logger.info(f"Extracting zip file to {self.config.unzip_dir}")
        except Exception as e:
            self.logger.error(f"Error extracting zip file to {self.config.unzip_dir}")
            self.logger.error(e)
            raise e


In [8]:
config = CofigurationManager()

data_ingestion_config = config.get_data_ingestion_config()

data_ingestion = DataIngestion(data_ingestion_config)

data_ingestion.download_data()
data_ingestion.extract_zip_file()

2025-02-07 18:25:27,959 - __main__ - INFO - 2573542362.py - 25 - File artifacts/data_ingestion/summarizer-data.zip/summarizer-data.zip already exists
2025-02-07 18:25:28,091 - __main__ - INFO - 2573542362.py - 43 - Extracting zip file to artifacts/data_ingestion
